In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
from torch.utils import data
from dataset import*
from config import settings
from torch.utils.tensorboard import SummaryWriter
from model import*
from train_test import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
transformers = {
        'year': MinMaxScaler(), # Normalize
        'weekday': None,
        'time': None, # Standardlize
        'weather': None
    }
train_df, transformers = load_data('train', total_sample=1000000, random_sample=settings.totalN, scaling_transformers=transformers)
test_df, transformers = load_data('test', random_sample=settings.totalN, scaling_transformers=transformers)

loaded csv file shape: (1000000, 8)
setting time info...
setting geo info...
counting net fare...
loaded test csv file shape: (9914, 7)
setting time info...
setting geo info...
counting fixed fee...


In [3]:
train_data_df = train_df.copy()
test_data_df = test_df.copy()
train_data_df.drop('fare_amount', axis=1, inplace=True)
train_data_df.drop('total_fixed_fees', axis=1, inplace=True)
test_data_df.drop('total_fixed_fees', axis=1, inplace=True)
test_data_df.drop('key', axis=1, inplace=True)

In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [5]:

# 假設你的目標欄位叫做'target'
X_train = train_data_df.drop('net_fare', axis=1)
y_train = train_data_df['net_fare']

# 建立XGBoost回歸模型
model = xgb.XGBRegressor(objective ='reg:squarederror', learning_rate = 0.01, max_depth = 5, n_estimators = 100)

# 訓練模型
model.fit(X_train, y_train)

# 用訓練好的模型來預測測試資料集
predictions = model.predict(test_data_df)


In [6]:
# 將預測結果轉換為DataFrame
total_fixed_fees = test_df['total_fixed_fees'].values
predictions_df = pd.DataFrame(predictions+total_fixed_fees, columns=['fare_amount'])


# 從測試數據集中提取'key'欄位
keys_df = test_df['key']

# 將'key'與預測結果組合成一個新的DataFrame
result_df = pd.concat([keys_df.reset_index(drop=True), predictions_df], axis=1)

In [7]:
result_df.to_csv('result.csv', index=False)